In [1]:
import sys
from pandas import *
import numpy as np
import matplotlib
from matplotlib import pyplot
import itertools as it
import matplotlib.backends.backend_pdf
import math
from matplotlib.pyplot import cm
from dateutil import parser
import scipy
from scipy.stats import mstats
import re
import matplotlib.dates as mdates
import datetime
import msprime as ms
import random
from scipy.stats import norm

In [2]:
def generate_haplotypes(var_dict,samp_size,seq_len):
    hap_list = ['' for x in range(samp_size)]
    for pos in range(seq_len):
        if(pos in var_dict.keys()):
            for ind,al in enumerate(var_dict[pos]):
                hap_list[ind] = ''.join([hap_list[ind],str(al)])
#         else:
#             for ind in range(samp_size):
#                 hap_list[ind] = ''.join([hap_list[ind],'0'])
    ind_haps_dict = {}
    haps_used = []
    x = int(samp_size/2)
    for n in range(x):
        curr_haps = []
        while (len(curr_haps) != 2):
            temp_seq_num = random.randint(0,(samp_size-1))
            if(temp_seq_num not in haps_used):
                curr_haps.append(hap_list[temp_seq_num])
                haps_used.append(temp_seq_num)
        ind_haps_dict[n] = curr_haps
    return ind_haps_dict

def generate_betas(num_inds,dist_type='normal'):
    if(dist_type == 'normal'):
        dist = np.random.normal(0,0.1,num_inds)
    return dist


def assign_genotype_index(samp_size,num_inds):
    ind_haps_dict = {}
    haps_used = []
    x = num_inds
#     x = int(samp_size/2)
    for n in range(x):
        curr_haps = []
        while (len(curr_haps) != 2):
            temp_seq_num = random.randint(0,(samp_size-1))
            if(temp_seq_num not in haps_used and temp_seq_num not in curr_haps):
                curr_haps.append(temp_seq_num)
                haps_used.append(temp_seq_num)
        ind_haps_dict[n] = curr_haps
    return ind_haps_dict



def estimate_pheno(genotypes,beta):
#     causal_pos = 0 #int(len(genotype[0])/2)
    full_phenovals = []
    for g in genotypes:
        try:
            c_al1 = float(g[0])
            c_al2 = float(g[1])
            full_phenovals.append((c_al1 + c_al2)*beta)
        except:
            print("Error!",g)
    return sum(full_phenovals)

def plot_phenodist(pheno_dict):
    pheno_df = DataFrame(pheno_dict.values(),index=pheno_dict.keys(),columns=['Pheno_value'])

    mu, std = norm.fit(pheno_df['Pheno_value'])

    pyplot.hist(pheno_df['Pheno_value'])

    xmin, xmax = pyplot.xlim()# min(pheno_df['Pheno_value']),max(pheno_df['Pheno_value'])
    x = np.linspace(xmin, xmax, 100)
#     p = (norm.pdf(x, mu, std))*(num_inds)
    p = (norm.pdf(x, mu, std))*(len(pheno_df)*2)
    pyplot.plot(x, p, 'k', linewidth=2)
    title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
    pyplot.title(title)
    pyplot.ylabel('Number of Individuals in Bin')
    pyplot.xlabel('Phenotype Value')
#     pyplot.savefig('/local3/jake/admix_simul/pheno_plot_test.png')
    pyplot.show()
    



In [3]:


def write_phenofile(outname,pheno_dict,popid_dict=None):
    pheno_file = open('{0}.phenotypes'.format(outname),'w')
    if(popid_dict == None):
        new_ids = []
        for i in range(len(pheno_dict.items())):
            new_ids.append(''.join(['ID',str(i)]))
    else:
        new_ids = []
        for i,pop in popid_dict.items():
            new_ids.append(''.join(['POP',str(pop),'ID',str(i),]))
    for ind,p in pheno_dict.items():
        pheno_file.write('{0}\t{1}\n'.format(new_ids[ind],p))
    pheno_file.close()

def run_pheno_simulation(samp_size,seq_len,L,num_individuals,causal_var_id=1):
    genotype_index_byinds = assign_genotype_index(samp_size,num_individuals) #randomly take the <samp_size> number of genomes simulated, and randomly assign to each individual 2 of them
#     causalgenotypes_byind = {x:[] for x in range(num_individuals)} #Each repetition, add the genotype from that rep (as a tuple) to this dictionary. Then we can just iterate through the list of values to generate the final phenotype
    causalgenotypes_byrep = {x:[] for x in range(L)}
    causalpositions_byrep = {x:0 for x in range(L)}
    fullgenotypes_byrep = {x:[] for x in range(L)}
    rep = 0
    while (rep < L):
        tree_sequence = ms.simulate(sample_size=samp_size, Ne=1e4, length=seq_len, recombination_rate=2e-8,mutation_rate=2e-8) 
        curr_causal_var = []
        curr_causal_pos = 0
        curr_full_vars_posgeno_dict = {}
        for variant in tree_sequence.variants():
            curr_full_vars_posgeno_dict[round(variant.site.position)] = list(variant.genotypes)
            if(variant.site.id == causal_var_id):
                curr_causal_var = list(variant.genotypes)
                curr_causal_pos = round(variant.site.position)

        if(len(curr_causal_var) != samp_size):
            print('no causal variants in rep {0}, redoing'.format(rep))
            rep -= 1
            continue
        causalpositions_byrep[rep] = curr_causal_pos
        curr_rep_genotypes = []
        for indiv,index in genotype_index_byinds.items():
            try:
                causalgenotypes_byind[indiv].append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
                curr_rep_genotypes.append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
            except:
                print(indiv,index)
        causalgenotypes_byrep[rep] = [curr_causal_pos,curr_rep_genotypes]
        
        curr_rep_fullgenos = {}
        for pos,geno in curr_full_vars_posgeno_dict.items():
            temp_fullgeno = []
            for indiv,index in genotype_index_byinds.items():
                try:
                    temp_fullgeno.append((geno[index[0]],geno[index[1]]))
                except:
                    print(indiv,index)
            curr_rep_fullgenos[pos] = temp_fullgeno
        fullgenotypes_byrep[rep] = curr_rep_fullgenos
        rep += 1

    phenotypes_byinds = {x:0 for x in range(num_individuals)}
    
    beta_list = generate_betas(num_inds=num_individuals)
    for i in range(num_individuals):
        curr_beta = beta_list[i]
        phenotypes_byinds[i] = estimate_pheno(causalgenotypes_byind[i],curr_beta)
        
    plot_phenodist(phenotypes_byinds)
    write_phenofile('test_phenos',phenotypes_byinds)
    
    return causalpositions_byrep,fullgenotypes_byrep
    
   
    

In [4]:
def run_geno_simulation(samp_size,seq_len,L,num_individuals,causal_var_id=1):
    genotype_index_byinds = assign_genotype_index(samp_size,num_individuals) #randomly take the <samp_size> number of genomes simulated, and randomly assign to each individual 2 of them
    causalgenotypes_byind = {x:[] for x in range(num_individuals)} #Each repetition, add the genotype from that rep (as a tuple) to this dictionary. Then we can just iterate through the list of values to generate the final phenotype
    causalgenotypes_byrep = {x:[] for x in range(L)}
    rep = 0
    while (rep < L):
        tree_sequence = ms.simulate(sample_size=samp_size, Ne=1e4, length=seq_len, recombination_rate=2e-8,mutation_rate=2e-8) 
        num_vars = 0
        for variant in tree_sequence.variants():
            num_vars += 1
        causal_var_id = temp_seq_num = random.randint(0,(num_vars-1))
        curr_causal_var = []
        curr_causal_pos = 0
        for variant in tree_sequence.variants():
            if(variant.site.id == causal_var_id):
                curr_causal_var = list(variant.genotypes)
                curr_causal_pos = round(variant.site.position)

        if(len(curr_causal_var) != samp_size):
            print('no causal variants in rep {0}, redoing'.format(rep))
            rep -= 1
            continue
        curr_rep_genotypes = []
        for indiv,index in genotype_index_byinds.items():
            try:
                curr_rep_genotypes.append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
#                 causalgenotypes_byind[indiv].append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
            except:
                print(indiv,index)
        causalgenotypes_byrep[rep] = [curr_causal_pos,curr_rep_genotypes]
        rep += 1
    return causalgenotypes_byrep



In [5]:
def write_genovcf(full_geno_dict,causal_pos_dict,outname,seq_len,window_spacer=1000):
#     full_len = len(full_geno_dict)*(seq_len + (window_spacer*2))
    full_len = len(full_geno_dict)*window_spacer
    header_string = '##fileformat=VCFv4.2 \n##source=tskit 0.2.2 \n##FILTER=<ID=PASS,Description="All filters passed"> \n##INFO=<ID=CS,Number=0,Type=Flag,Description="SNP Causal to Phenotype"> \n##contig=<ID=1,length={0}>\n##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype"> \n'.format(full_len)
    vcf_file = open('/local3/jake/admix_simul/testing/{0}.vcf'.format(outname),'w')
    vcf_file.write(header_string)
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t')
    new_ids = []
    for i in range(len(full_geno_dict[0][causal_pos_dict[0]])):
        curr_nid = ''.join(['ID',str(i)])
        new_ids.append(curr_nid)
        vcf_file.write('{0}\t'.format(curr_nid))
    vcf_file.write('\n')
    for rep,geno_dict in full_geno_dict.items():
        is_rep_causalsnp = False
        for pos,genos in geno_dict.items():
            try:
                curr_pos = (int(rep)*window_spacer)+pos
                if(pos == causal_pos_dict[rep]):
                    is_rep_causalsnp = True
                    vcf_file.write('1\t{0}\t.\tA\tG\t.\tPASS\tCS\tGT\t'.format(curr_pos))
                else:
                    vcf_file.write('1\t{0}\t.\tA\tG\t.\tPASS\t.\tGT\t'.format(curr_pos))
                for g in genos:
                    vcf_file.write('{0}|{1}\t'.format(g[0],g[1]))
                vcf_file.write('\n')
            except:
                print(rep,genos)
    
        

In [6]:
def get_input_migrationmatrix(filename,num_epochs):
    mfile = open(filename,'r')
    migmats_byepoch = [[] for x in range(num_epochs)]
    for line in mfile:
        if(line[0] == '#'):
            curr_migmat = int(line[1])
        else:
            curr_mm = np.array([float(f) for f in line.split('\n')[0].split('\t')])
            migmats_byepoch[curr_migmat].append(curr_mm)
    mfile.close()
    return migmats_byepoch
def read_input_file(filename):
    input_df = read_csv('{0}.epochs'.format(filename),sep='\t')
    num_theta = len(input_df)
    theta_true = []
    sample_scheme_byepoch = []
    migmat_fromfile = None
    try:
        mm_file = '{0}.migration_matrix'.format(filename)
        migmat_fromfile = get_input_migrationmatrix(filename,num_theta)
    except:
        pass
    # Set the demographic model parameters (theta)...
    for epoch,t in input_df.iterrows():
        Np = len(t['pop_scheme'].split(','))               # Num pops
        pop_sizes = [int(x) for x in t['pop_scheme'].split(',')]    # Num of samples per population in the observed data
        sample_scheme = [int(x) for x in t['pop_scheme'].split(',')]   # Num samples drawn from each pop for likelihood

        # pop sizes
        N_base = 1e4    # Number of diploids in the population
        Ne0 = np.ones(Np)*N_base
        
        if(migmat_fromfile is None):
            # Set migration rates
            M_base = float(t['migration_rate'])
            m = np.zeros((Np,Np))
            for i in range(Np):
                for j in range(Np):
                    if j!=i:
                        m[i,j] = M_base/(2*(Np-1))
            theta_true.append([float(t['time']),[Ne0,m]])
        else:
            theta_true.append([float(t['time']),migmat_fromfile[epoch]])
        sample_scheme_byepoch.append(pop_sizes)

    return theta_true,sample_scheme_byepoch


#Reads in a file that has the epochs that we want to simulate. The file needs to be in a tab-seperated file, with a line per epoch, each one followed by a migration matrix outlining the migration values for each population pair
#Each line that indicates an epoch should start with a '#' symbol, followed immediately by the time of the start of the epoch, then a tab, then the population sizes seperated by commas (for example: 4,4,5 for 3 populations of that number of individuals)
#Following the line with the time/population sizes, the migration matrix which is of size NxN, where N=number of populations. The diagonal of the matrix should be zeros.
#   filename (str): name of the input file to read
#returns:
#   theta_true: description can be found in the run_
def read_input_file_full(filename):
    infile = open(filename,'r')
    theta_true = []
    sample_scheme_byepoch = []
    curr_epoch = -1
    migmats_byepoch = {}
    times_byepoch = []
    for line in infile:
        if(line[0] == '#'):
            curr_epoch += 1
            curr_time = line.split('\n')[0][1:].split('\t')[0]
            times_byepoch.append(float(curr_time))
            curr_popscheme = line.split('\n')[0][1:].split('\t')[1]
            pop_sizes = [int(x) for x in curr_popscheme.split(',')]    # Num of samples per population in the observed data
            sample_scheme_byepoch.append(pop_sizes)
        else:
            curr_mm = [float(f) for f in line.split('\n')[0].split('\t')]
            if(curr_epoch not in migmats_byepoch.keys()):
                migmats_byepoch[curr_epoch] = []
            migmats_byepoch[curr_epoch].append(curr_mm)
    # pop sizes
    N_base = 1e4    # Number of diploids in the population
    
    for e in range(len(sample_scheme_byepoch)):
        Ne0 = np.ones(len(sample_scheme_byepoch[e]))*N_base
        theta_true.append([times_byepoch[e],[Ne0,np.array(migmats_byepoch[e])]])

    return theta_true,sample_scheme_byepoch


In [7]:
def write_phenovcf(pheno_dict,outname,seq_len,pop_dict=None):
    header_string = '##fileformat=VCFv4.2 \n##source=tskit 0.2.2 \n##FILTER=<ID=PASS,Description="All filters passed"> \n##contig=<ID=1,length=1000>\n##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype"> \n'
    vcf_file = open('/local3/jake/admix_simul/testing/{0}.vcf'.format(outname),'w')
    vcf_file.write(header_string)
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t')
    if(pop_dict == None):
        new_ids = []
        for i in range(len(geno_dict[0][1])):
            curr_nid = ''.join(['ID',str(i)])
            new_ids.append(curr_nid)
            vcf_file.write('{0}\t'.format(curr_nid))
        vcf_file.write('\n')
    else:
        new_ids = []
        for i,pop in popid_dict.items():
            curr_nid = ''.join(['POP',str(pop),'ID',str(i),])
            new_ids.append(curr_nid)
            vcf_file.write('{0}\t'.format(curr_nid))
        vcf_file.write('\n')
    for rep,genos in geno_dict.items():
        curr_pos = (int(rep)*seq_len)+genos[0]
        vcf_file.write('1\t{0}\t.\tA\tG\t.\tPASS\t.\tGT\t'.format(curr_pos))
        for g in genos[1]:
            vcf_file.write('{0}|{1}\t'.format(g[0],g[1]))
        vcf_file.write('\n')

In [8]:
def get_indtopop_dict(genoind_bypops):
    indpop_dict = {}
    prev_pn = 0
    for num,p in enumerate(genoind_bypops.items()):
        for i,g in enumerate(p[1]):
            indpop_dict[(prev_pn+i)] = p[0]
        prev_pn += len(p[1])
    return indpop_dict

def get_ind_genoindex_multipop(genoind_bypops):
    ind_genoindex_dict = {}
    prev_pn = 0
    for num,p in enumerate(genoind_bypops.items()):
        for i,g in enumerate(p[1]):
            ind_genoindex_dict[(prev_pn+i)] = g
        prev_pn += len(p[1])
    return ind_genoindex_dict

def assign_pop_inds(pop_sizes):
    pop_assign = []  # row index = pop index. value = list of indices of samples assigned to each pop.
    j = 0
    for i in range(len(pop_sizes)):
        Ni = pop_sizes[i]
        indices = list(range(j,j+Ni))
        pop_assign.append(indices)
        j+=Ni
    return pop_assign
def assign_popdict(pop_scheme):
    full_sampsize = sum(pop_scheme)
    popindex_byind = {x:0 for x in range(full_sampsize)}
    for pop,size in enumerate(pop_scheme):
        for x in range(size):
            popindex_byind[(x + (size*pop))] = pop
    return popindex_byind

#Take the genotypes that are simulated, and assign two of the indexes to each individual.
#samp_sizes: number of individuals that you want to sample, an array of length P
#pop_sizes: Total number of genotypes in each population that you are simulating, an array of length P. Each entry must be at least double the entry in samp_sizes
def assign_genotype_index_multipop(samp_sizes,pop_sizes):
    for i,s in enumerate(samp_sizes):
        if(pop_sizes[i] < (2*s)):
            print('number of genotypes in population {0} is not at least double the number of individuals you are simulating! {1} < {2}'.format(i,pop_sizes[i],(s*2)))
            return -1
    ind_haps_dict_bypop = {x:[] for x in range(len(pop_sizes))}
    x = sum(pop_sizes)
    prev_indstart = 0
    for n,p in enumerate(pop_sizes):
        num_inds_tosample = samp_sizes[n]
        curr_popinds = []
        for x in range(p):
            curr_popinds.append(x + prev_indstart)
        
        print(n,p,num_inds_tosample,curr_popinds)
        popinds_used = []
        temp_popinds = []
        for x in range(num_inds_tosample):
            curr_haps = []
            while (len(curr_haps) != 2):
                temp_seq_num = random.randint(min(curr_popinds),max(curr_popinds))
                if(temp_seq_num not in popinds_used and temp_seq_num not in curr_haps):
                    curr_haps.append(temp_seq_num)
                    popinds_used.append(temp_seq_num)
            temp_popinds.append(curr_haps)
        ind_haps_dict_bypop[n] = temp_popinds
        prev_indstart += p
    return ind_haps_dict_bypop


In [52]:
# test_theta,test_sampsch = read_input_file_full_v1('/local3/jake/admix_simul/testing/test_inputfile.migration_events')
# test_theta,test_sampsch = read_input_file_full('/local3/jake/admix_simul/testing/test_inputfile.migration_matrix')
test_theta,test_sampsch = read_input_file_full_v1('/local3/jake/admix_simul/testing/single_massmig_event/input_file.migr_events.v2')



In [57]:
test_theta

[['ooafrica', [100, 0, 0]],
 ['mass', 120.0, [array([12300., 12300., 12300.]), array([[1. , 0. , 0.5]])]]]

In [130]:
mu=1e-7
r=2e-8  
cpos_pheno_byinds,pgenos_byinds = run_pheno_simulation_multipops(test_theta,1000,25,test_sampsch,r,mu,'/local3/jake/admix_simul/testing/test_output_massmig','normal')


Starting epoch 0 replicates
0 8 4 [0, 1, 2, 3, 4, 5, 6, 7]
1 8 4 [8, 9, 10, 11, 12, 13, 14, 15]
2 8 4 [16, 17, 18, 19, 20, 21, 22, 23]
there is only one epoch, starting now using theta [['mrate', 0.0, [array([10000., 10000., 10000.]), array([[0.    , 0.0001, 0.01  ],
       [0.0001, 0.    , 0.001 ],
       [0.01  , 0.001 , 0.    ]])]]]
replicate trees created; iterating through genotypes now
finished iterating through trees, starting the phenotype calculations
Starting epoch 1 replicates
0 8 4 [0, 1, 2, 3, 4, 5, 6, 7]
1 14 7 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
2 14 7 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Epoch 1 has no migration, skipping this epoch due to memory constraints
Starting epoch 2 replicates
0 8 4 [0, 1, 2, 3, 4, 5, 6, 7]
1 14 7 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
2 14 7 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Epoch 2 has no migration, skipping this epoch due to memory constraints
Starting epoch 3 repl

In [10]:
def thetas_toskip(theta):
    bad_theta_nums = []
    for epoch in range(len(theta)):
        curr_theta = theta[:epoch+1]
        if(curr_theta[-1][0] == 'mass'):
            bad_theta_nums.append(epoch)
        elif(set([all(x == 0) for x in curr_theta[-1][2][1]]) == {True}):
            bad_theta_nums.append(epoch)
    return bad_theta_nums

In [128]:
# thetas_toskip(test_theta)
test_theta[-1][0]

'mrate'

In [17]:
#theta: the list of epochs that we want to simulate. Each entry is equal to one epoch; each entry has:
#  0: Starting time of the epoch
#  1: List with two items: Total Population size at start of epoch, and migration matrix
#    Migration Matrix: NxN (where N = Number of Populations, Np), Rate of migration between each population
#sample_nums: Array of population sizes, of length Np; the number of samples drawn from the population
#L: Length of the sequence we will be simulating
#r: Recombination rate, constant accross all populations
#mu: Mutation rate, constant accross all populations
#R: Number of replicates; msprime will run R simulations and return an iterator over all trees created
#Updated: Return list of ts_replicates; each entry will be the simulations for each epoch
def run_msprime_tskit(theta,sample_nums,L,r,mu,R):
    ts_replicates = None
    
    Np = len(sample_nums)
#     print('samples to be drawn for populations: {0}'.format(sample_nums))
    init_pop_configs = [ms.PopulationConfiguration(sample_size=sample_nums[i], initial_size=theta[0][2][0][i]) for i in range(Np)]

    if(theta[0][0] == 'mrate'):
        init_mig = theta[0][2][1]
    elif(theta[0][0] == 'mass'):
        init_mig = None
        init_demoevents = []
        for m in theta[0][2][1]:
            init_demoevents.append(ms.MassMigration(time=0, source=m[0], destination=m[1], proportion=m[2]))
    K = len(theta)  # K = number of epochs
    if K > 1:
        # There is more than one epoch, so must set the non-initial epochs as demographic events
        demo_events = []
        for k in range(1,K):
            theta_type,t_k,theta_k = theta[k]
            Ne = theta_k[0]
            if(theta_type == 'mrate'):
                mig = theta_k[1]
                for i in range(Np):
                    # Set the Ne
                    demo_events.append(ms.PopulationParametersChange(population=i,time=t_k,initial_size=Ne[i]))

                    # Set the migration rates
                    for j in range(Np):
                        if j!=i:
                            demo_events.append(ms.MigrationRateChange(time=t_k,rate=mig[i,j],matrix_index=tuple([i,j])))
            elif(theta_type == 'mass'):
                massmig = theta_k[1]
                for m in massmig:
                    demo_events.append(ms.MassMigration(time=t_k, source=m[0], destination=m[1], proportion=m[2]))
            else:
                print('Epoch events should only be "mass" or "mrate"!')
                return -1
#         print(init_pop_configs)
        ts_replicates = ms.simulate(
            length=L,
            recombination_rate=r,
            population_configurations=init_pop_configs,
            migration_matrix = init_mig,
            demographic_events=demo_events,
            num_replicates=R,
            mutation_rate = mu
        )
    else:
        print('there is only one epoch, starting now using theta {0}'.format(theta))
        # There is only the initial epoch. 
        if(init_mig is None): #If the first epoch event is a mass migration event, then there's no migration matrix to provide
            ts_replicates = ms.simulate(
            length=L,
            recombination_rate=r,
            population_configurations=init_pop_configs,
            demographic_events = init_demoevents,
            num_replicates=R,
            mutation_rate = mu)
        else:
            ts_replicates = ms.simulate(
                length=L,
                recombination_rate=r,
                population_configurations=init_pop_configs,
                migration_matrix = init_mig,
                num_replicates=R,
                mutation_rate = mu
            )
    return ts_replicates


In [12]:
# Fixed model parameters...
mu=1e-7                                  # mutation rate per site per generation per lineage
r=2e-8                                   # recombination rate per site per generation per lineage
Lobs = int(2e4)                          # The total number of sites in the observed sequence data
R=20                                     # Num of replicate ARGs to run per model
L=Lobs                                   # size of each ARG window 
windows = [[0,L]]                        # List of the windows to use
draws_max = 1                            # Maximum number of sample draws to use
tol = 0.5                                # Tolerance in fraction of mutational patterns that don't match the ARG tree among usable sites
order='natural'                             # The sample label order to use. 'natural'= do nothing. 'WPDO'= put samples in Within-Population Derived-allele Order

# Set the demographic model parameters (theta)...
Np = 3                                  # Num pops
pop_sizes = [4 for i in range(Np)]      # Num of samples per population in the observed data
sample_scheme = [4 for i in range(Np)]  # Num samples drawn from each pop for likelihood

# pop sizes
N_base = 1e4    # Number of diploids in the population
Ne0 = np.ones(Np)*N_base

# Set migration rates
M_base = 1e-4
m0 = np.zeros((Np,Np))
m1 = m0.copy()
for i in range(Np):
    for j in range(Np):
        if j!=i:
            m1[i,j] = M_base/(2*(Np-1))
            
phi0 = [Ne0,m0]
phi1 = [Ne0,m1]
t1 = 1e4
theta_true=[[0.,phi0],[t1,phi1]]

pops = [i for i in range(Np)]
n = np.sum(sample_scheme)
N = np.sum(pop_sizes)
pop_assign = []  # row index = pop index. value = list of indices of samples assigned to each pop.
j = 0
for i in range(Np):
    Ni = pop_sizes[i]
    indices = list(range(j,j+Ni))
    pop_assign.append(indices)
    j+=Ni

# Generate the fake seq data from the true model, S
# S = simulate_sequence_data(theta_true,pop_sizes,Lobs,r,mu)



In [74]:
# test_tsreps = run_msprime_tskit(theta_true,sample_scheme,1000,r,mu,R)
# run_pheno_simulation_multipops(theta_true,1000,50,3,4)
theta = theta_true
seq_len = 1000
reps = 50
r = 2e-8 #recombination rate
mu = 2e-8 #mutation rate
num_pops = 3
num_individuals = 4

pop_sizes = [(num_individuals*2) for i in range(num_pops)] # Num individuals simulated, equal to twice the number of individuals, since each person is diploid
sample_scheme = [num_individuals for i in range(num_pops)] # Num samples drawn from each pop for likelihood
full_sampsize = sum(sample_scheme)

genotype_index_bypops = assign_genotype_index_multipop(sample_scheme,pop_sizes) #randomly take the <samp_size> number of genomes simulated, and randomly assign to each individual 2 of them
genotype_index_byinds = assign_genotype_index(sum(pop_sizes),full_sampsize)
causalgenotypes_byrep = {x:[] for x in range(reps)}
causalpositions_byrep = {x:0 for x in range(reps)}
fullgenotypes_byrep = {x:[] for x in range(reps)}


phenos_byepoch = []
genos_byepoch = []
causalpos_byepoch = []
popindex_byind = {x:0 for x in range(full_sampsize)}
for pop,size in enumerate(sample_scheme):
    for x in range(size):
        popindex_byind[(x + (size*pop))] = pop

for epoch in range(len(theta)):
    print('Starting epoch {0} replicates'.format(epoch+1))
#     print(epoch,len(theta[:epoch+1]))
    curr_theta = theta[:epoch+1]
    new_sampscheme = [8 for x in range(num_pops)]
    tsreps = run_msprime_tskit_updated(curr_theta,pop_sizes,seq_len,r,mu,reps)
    for rep,tree_sequence in enumerate(tsreps): 
        num_vars = 0
        for variant in tree_sequence.variants():
            num_vars += 1
        causal_var_id = random.randint(0,(num_vars-1))
        curr_causal_var = []
        curr_causal_pos = 0
        curr_full_vars_posgeno_dict = {}
        for variant in tree_sequence.variants():
#             print(len(list(variant.genotypes)))
            curr_full_vars_posgeno_dict[round(variant.site.position)] = list(variant.genotypes)
            if(variant.site.id == causal_var_id):
                curr_causal_var = list(variant.genotypes)
                curr_causal_pos = round(variant.site.position)

        if(len(curr_causal_var) != (full_sampsize*2)):
            print('not enough causal variants in rep {0}'.format(rep))
            curr_causal_var = [0 for x in range((full_sampsize*2))]
            curr_full_vars_posgeno_dict[0] = [0 for x in range((full_sampsize*2))]
        causalpositions_byrep[rep] = curr_causal_pos

        curr_rep_genotypes = []
        for indiv,index in genotype_index_byinds.items():
            try:
                curr_rep_genotypes.append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
            except:
                print(indiv,index,len(curr_causal_var))
        causalgenotypes_byrep[rep] = [curr_causal_pos,curr_rep_genotypes]

        curr_rep_fullgenos = {}
        for pos,geno in curr_full_vars_posgeno_dict.items():
            temp_fullgeno = []
            for indiv,index in genotype_index_byinds.items():
                try:
                    temp_fullgeno.append((geno[index[0]],geno[index[1]]))
                except:
                    print(rep,indiv,index,len(geno))
            curr_rep_fullgenos[pos] = temp_fullgeno
        fullgenotypes_byrep[rep] = curr_rep_fullgenos
        
    causalgenotypes_byind = {x:[] for x in range(full_sampsize)}
    for pos,genos in causalgenotypes_byrep.items():
        for num,g in enumerate(genos[1]):
            causalgenotypes_byind[num].append(g)
    beta = 'normal'
    phenotypes_byinds = {x:0 for x in range(full_sampsize)}
    beta_list = generate_betas(num_inds=full_sampsize,dist_type=beta)
    for i in range(full_sampsize):
        curr_beta = beta_list[i]
        phenotypes_byinds[i] = estimate_pheno(causalgenotypes_byind[i],curr_beta)
    phenos_byepoch.append(phenotypes_byinds)
    genos_byepoch.append(fullgenotypes_byrep)
    causalpos_byepoch.append(causalpositions_byrep)
    

Starting epoch 1 replicates
Starting epoch 2 replicates


In [13]:
def write_genovcf(full_geno_dict,causal_pos_dict,outname,num_inds,seq_len,window_spacer=1000,popid_dict=None):
    use_diff_chrms = False
    if(window_spacer == 'chrm'):
        window_spacer = 0
        use_diff_chrms = True
    full_len = len(full_geno_dict)*(window_spacer+seq_len)
    header_string = '##fileformat=VCFv4.2 \n##source=tskit 0.2.2 \n##FILTER=<ID=PASS,Description="All filters passed"> \n##INFO=<ID=CS,Number=0,Type=Flag,Description="SNP Causal to Phenotype"> \n##contig=<ID=1,length={0}>\n##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype"> \n'.format(full_len)
    vcf_file = open('{0}.vcf'.format(outname),'w')
    vcf_file.write(header_string)
    vcf_file.write('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t')
    if(popid_dict == None):
        new_ids = []
        for i in range(len(geno_dict[0][1])):
            curr_nid = ''.join(['ID',str(i)])
            new_ids.append(curr_nid)
            vcf_file.write('{0}\t'.format(curr_nid))
        vcf_file.write('\n')
    else:
        new_ids = []
        for i,pop in popid_dict.items():
            curr_nid = ''.join(['POP',str(pop),'ID',str(i),])
            new_ids.append(curr_nid)
            vcf_file.write('{0}\t'.format(curr_nid))
        vcf_file.write('\n')
    for rep,geno_dict in full_geno_dict.items():
        is_rep_causalsnp = False
        for pos,genos in geno_dict.items():
            try:
                if(use_diff_chrms == True):
                    curr_pos = (int(rep)*(window_spacer+seq_len))+pos
                    if(pos == causal_pos_dict[rep]):
                        is_rep_causalsnp = True
                        vcf_file.write('{0}\t{1}\t.\tA\tG\t.\tPASS\tCS\tGT\t'.format(rep,curr_pos))
                    else:
                        vcf_file.write('{0}\t{1}\t.\tA\tG\t.\tPASS\t.\tGT\t'.format(rep,curr_pos))
                    for g in genos:
                        vcf_file.write('{0}|{1}\t'.format(g[0],g[1]))
                    vcf_file.write('\n')
                else:
                    curr_pos = (int(rep)*(window_spacer+seq_len))+pos
                    if(pos == causal_pos_dict[rep]):
                        is_rep_causalsnp = True
                        vcf_file.write('1\t{0}\t.\tA\tG\t.\tPASS\tCS\tGT\t'.format(curr_pos))
                    else:
                        vcf_file.write('1\t{0}\t.\tA\tG\t.\tPASS\t.\tGT\t'.format(curr_pos))
                    for g in genos:
                        vcf_file.write('{0}|{1}\t'.format(g[0],g[1]))
                    vcf_file.write('\n')

            except:
                print(rep,genos)

In [14]:
def thetas_toskip(theta):
    bad_theta_nums = []
    for epoch in range(len(theta)):
        curr_theta = theta[:epoch+1]
        if(set([all(x == 0) for x in curr_theta[-1][1][1]]) == {True}):
            bad_theta_nums.append(epoch)
    return bad_theta_nums

def init_thetas_toskip(theta):
    bad_theta_nums = []
    is_init_epoch = True
    for epoch in range(len(theta)):
        curr_theta = theta[:epoch+1]
        if(is_init_epoch == True and set([all(x == 0) for x in curr_theta[-1][1][1]]) == {True}):
            bad_theta_nums.append(epoch)
        else:
            is_init_epoch = False
    return bad_theta_nums

In [15]:
#Runs the phenotype-generating simulation, for multiple populations and multiple epochs
#    theta: list of variables, each entry indicates an epoch. See the description for run_msprime_tskit() to see the full details of this variable
#    seq_len (int): length of the sequence that will be simulated
#    reps (int): number of replicates to simulate
#    pop_schemes (list): list of ints, one per population that will be simulated, indicating the number of individuals that will have 2 genotypes assigned to each
#    r (float): recombination rate, provided to the simulate function
#    mu (float): mutation rate, provided to the simulate function
#    outname (string): name of the file that will be created with the phenotype, one per epoch that is provided, as <outname>.epoch<num_epoch>.phenotypes
#    beta (string): method of choosing the beta value that is assigned to each individual. The only kind of distribution that is implemented so far
#Returns a 2 lists of dictionaries; per epoch. 
#    causalpos_byepoch: for each replicate in the epoch, contains the position that was used to calculate the phenotype. Used to indicate in the output vcf file which SNP is the causal one
#    genos_byepoch: for each replicate in each epoch, has a dictionary containing the full genotypes for all variants
def run_pheno_simulation_multipops(theta,seq_len,reps,pop_schemes,r,mu,outname,beta='normal'):
    
    genos_byepoch = []
    causalpos_byepoch = []
    epochs_toskip = thetas_toskip(theta)
    for epoch in range(len(theta)):
        print('Starting epoch {0} replicates'.format(epoch))

        pop_sizes = [(i*2) for i in pop_schemes[epoch]] # Num individuals simulated, equal to twice the number of individuals, since each person is diploid
        sample_scheme = [i for i in pop_schemes[epoch]] # Num samples drawn from each pop for likelihood
        full_sampsize = sum(sample_scheme)
        # print(pop_sizes,sample_scheme)
        genotype_index_bypops = assign_genotype_index_multipop(sample_scheme,pop_sizes) #randomly take the <samp_size> number of genomes simulated, and randomly assign to each individual 2 of them
        genotype_index_byinds = get_ind_genoindex_multipop(genotype_index_bypops)
        # print(genotype_index_byinds)
        causalgenotypes_byrep = {x:[] for x in range(reps)}
        causalpositions_byrep = {x:0 for x in range(reps)}
        fullgenotypes_byrep = {x:[] for x in range(reps)}


        curr_theta = theta[:epoch+1]
        if(epoch in epochs_toskip):
            print('Epoch {0} has no migration, skipping this epoch due to memory constraints'.format(epoch))
            continue
        # print('epoch {0} theta = {1}'.format(epoch,curr_theta))
        tsreps = run_msprime_tskit_v1(curr_theta,pop_sizes,seq_len,r,mu,reps)
        # pdb.set_trace()
        print('replicate trees created; iterating through genotypes now')

        for rep,tree_sequence in enumerate(tsreps): 
            # pdb.set_trace()
            # print('starting replicate {0}'.format(rep))
            num_vars = 0
            for variant in tree_sequence.variants():
                num_vars += 1
            causal_var_id = random.randint(0,(num_vars-1))
            curr_causal_var = []
            curr_causal_pos = 0
            curr_full_vars_posgeno_dict = {}
            for variant in tree_sequence.variants():
                curr_full_vars_posgeno_dict[round(variant.site.position)] = list(variant.genotypes)
                if(variant.site.id == causal_var_id):
                    curr_causal_var = list(variant.genotypes)
                    curr_causal_pos = round(variant.site.position)

            if(len(curr_causal_var) != (full_sampsize*2)):
                print('not enough causal variants in rep {0}'.format(rep))
                curr_causal_var = [0 for x in range((full_sampsize*2))]
                curr_full_vars_posgeno_dict[0] = [0 for x in range((full_sampsize*2))]
            causalpositions_byrep[rep] = curr_causal_pos

            curr_rep_genotypes = []
            for indiv,index in genotype_index_byinds.items():
                try:
                    curr_rep_genotypes.append((curr_causal_var[index[0]],curr_causal_var[index[1]]))
                except:
                    print(indiv,index,len(curr_causal_var))
            causalgenotypes_byrep[rep] = [curr_causal_pos,curr_rep_genotypes]

            curr_rep_fullgenos = {}
            for pos,geno in curr_full_vars_posgeno_dict.items():
                temp_fullgeno = []
                for indiv,index in genotype_index_byinds.items():
                    try:
                        temp_fullgeno.append((geno[index[0]],geno[index[1]]))
                    except:
                        print(rep,indiv,index,len(geno))
                curr_rep_fullgenos[pos] = temp_fullgeno
            fullgenotypes_byrep[rep] = curr_rep_fullgenos

        print('finished iterating through trees, starting the phenotype calculations')
        causalgenotypes_byind = {x:[] for x in range(full_sampsize)}
        for pos,genos in causalgenotypes_byrep.items():
            for num,g in enumerate(genos[1]):
                causalgenotypes_byind[num].append(g)

        beta = 'normal'
        phenotypes_byinds = {x:0 for x in range(full_sampsize)}
        beta_list = generate_betas(num_inds=reps,dist_type=beta)
        for i in range(full_sampsize):
            # curr_beta = beta_list[i]
            phenotypes_byinds[i] = estimate_pheno(causalgenotypes_byind[i],beta_list)
        # phenos_byepoch.append(phenotypes_byinds)
#         write_phenofile('{0}.epoch{1}'.format(outname,epoch),phenotypes_byinds)
        
        genos_byepoch.append(fullgenotypes_byrep)
        causalpos_byepoch.append(causalpositions_byrep)
    
    return causalpos_byepoch,genos_byepoch


In [77]:
def read_input_file_full_v1(filename):
    infile = open(filename,'r')
    theta_true = []
    sample_scheme_byepoch = []
    curr_epoch = -1
    migmats_byepoch = {}
    times_byepoch = []
    curr_epochtype = ''
    massmigs_byepoch = {}
    demoevs_byepoch = {}
    ooafrica_sampscheme = []
    for line in infile:
        if(line[0] == '#'):
            curr_epoch += 1
            cline = line.split('\n')[0][1:].split('\t')
            curr_epochtype = cline[0]
            if(curr_epochtype == 'ooafrica'):
                curr_popscheme = cline[1]
                curr_epoch -= 1
                ooafrica_sampscheme = [int(x) for x in curr_popscheme.split(',')]    # Num of samples per population in the observed data
                # sample_scheme_byepoch.append(ooafrica_sampscheme)
            else:
                curr_time = cline[1]
                times_byepoch.append(float(curr_time))
                curr_popscheme = cline[2]
                pop_sizes = [int(x) for x in curr_popscheme.split(',')]    # Num of samples per population in the observed data
                sample_scheme_byepoch.append(pop_sizes)
        else:
            curr_mm = [float(f) for f in line.split('\n')[0].split('\t')]
            if(curr_epoch not in demoevs_byepoch.keys()):
                demoevs_byepoch[curr_epoch] = [curr_epochtype]
            demoevs_byepoch[curr_epoch].append(curr_mm)

#     if(args.ooafrica):
    
    # pop sizes
    N_base = 12300    # Number of diploids in the population; based on the N_AF from the tutorial Out_Of_Africa model
    for e in range(len(sample_scheme_byepoch)):
        Ne0 = np.ones(len(sample_scheme_byepoch[e]))*N_base
        theta_true.append([demoevs_byepoch[e][0],times_byepoch[e],[Ne0,np.array(demoevs_byepoch[e][1:])]])
    if(len(ooafrica_sampscheme) > 0):
        theta_true.insert(0,['ooafrica',ooafrica_sampscheme,[]])
        sample_scheme_byepoch.insert(0,ooafrica_sampscheme)
    return theta_true,sample_scheme_byepoch




In [78]:
test_theta,test_sampsch = read_input_file_full_v1('/local3/jake/admix_simul/testing/single_massmig_event/input_file.migr_events.v2')



In [82]:
ooa_pop_configs, ooa_mig, ooa_demoevents = outofafrica_model_parameters(test_sampsch[0])

920.0


In [85]:
for d in ooa_demoevents:
    print(d)

Mass migration: Lineages moved with probability 1.0 backwards in time with source 2 & dest 1
                     (equivalent to migration from 1 to 2 forwards in time)
Migration rate change to 0 everywhere
Migration rate change for (0, 1) to 0.00015
Migration rate change for (1, 0) to 0.00015
Population parameter change for 1: initial_size -> 1861 growth_rate -> 0 
Mass migration: Lineages moved with probability 1.0 backwards in time with source 1 & dest 0
                     (equivalent to migration from 0 to 1 forwards in time)
Population parameter change for 0: initial_size -> 7300 


In [864]:
popindex_byind = {x:0 for x in range(full_sampsize)}
for pop,size in enumerate(sample_scheme):
    for x in range(size):
        popindex_byind[(x + (size*pop))] = pop

{0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 2, 9: 2, 10: 2, 11: 2}

In [132]:
g[0]

{5: [(0, 1),
  (0, 0),
  (1, 0),
  (1, 0),
  (0, 1),
  (0, 1),
  (0, 1),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (1, 1)],
 184: [(0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (0, 1),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (1, 1)],
 244: [(0, 1),
  (0, 0),
  (0, 0),
  (1, 0),
  (0, 1),
  (0, 1),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (1, 1)],
 340: [(0, 0),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 357: [(0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 464: [(0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (1, 0),
  (1, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0)],
 539: [(0, 0),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0)],
 547: [(0, 0),
  (0, 0),
  (1, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 1),
  (0, 0),
  (0, 0),
  (0, 0),
  (0, 0),
  (0,

In [22]:
# write_genovcf(g[0],c[0],'/local3/jake/admix_simul/testing/test_multipop_geno',full_sampsize,seq_len=1000,popid_dict=popindex_byind)
test_theta = read_input_file_full('/local3/jake/admix_simul/testing/test_inputfile.migration_matrix')
c,g = run_pheno_simulation_multipops(test_theta[0],1000,5,test_theta[1],2e-8,2e-8,'/local3/jake/admix_simul/testing/test_multipop_pheno.v3')



Starting epoch 0 replicates
0 8 4 [0, 1, 2, 3, 4, 5, 6, 7]
1 8 4 [8, 9, 10, 11, 12, 13, 14, 15]
2 8 4 [16, 17, 18, 19, 20, 21, 22, 23]
[[0.     0.0001 0.01  ]
 [0.0001 0.     0.001 ]
 [0.01   0.001  0.    ]]
there is only one epoch, starting now using theta [[0.0, [array([10000., 10000., 10000.]), array([[0.    , 0.0001, 0.01  ],
       [0.0001, 0.    , 0.001 ],
       [0.01  , 0.001 , 0.    ]])]]]
replicate trees created; iterating through genotypes now
finished iterating through trees, starting the phenotype calculations
Starting epoch 1 replicates
0 8 4 [0, 1, 2, 3, 4, 5, 6, 7]
1 14 7 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
2 14 7 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Epoch 1 has no migration, skipping this epoch due to memory constraints
Starting epoch 2 replicates
0 10 5 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1 12 6 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
2 14 7 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
[[0.     0.0001 0.01  ]
 [0.

In [14]:
m_AF_AS = 3e-5 #migration rate of african to non-african
m_EU_AS = 9.6e-5 #migration rate of non-african to african

time_migration = 3000

0
2
3
4


In [ ]:
phenotypes_byinds = {x:0 for x in range(full_sampsize)}
beta_list = generate_betas(num_inds=reps,dist_type=beta)
for i in range(full_sampsize):
    curr_beta = beta_list[i]
    phenotypes_byinds[i] = estimate_pheno(causalgenotypes_byind[i],curr_beta)

In [10]:
def create_sample_draws(draws_max,pop_assign,sample_scheme):
    """
    Compute draws of the observed samples to use for likelihood calculation
    """
    draws = []
    
    n = np.sum(sample_scheme)
    Np = len(sample_scheme)
    draws = []
    while (len(draws) < draws_max):
        draw = []
        for i in range(Np):
            ni = sample_scheme[i]
            samples_i = sorted(np.random.choice(pop_assign[i],size=ni,replace=False))
            draw.extend(samples_i)
            
        draw_tup = tuple(draw)
        if draw_tup not in draws:
            draws.append(draw_tup)
    return draws

def simulate_sequence_data(theta,sample_nums,L,r,mu):
    # Does not incorporate recombination maps. 
    S = []  # To return an L x N array, where N=sum(sample_nums)
    
    N = np.sum(sample_nums)
    print('Target number of sites in the seq:',L)    
    S = np.zeros((L,N),dtype=int)
    
    ts_replicates = run_msprime_tskit(theta,sample_nums,L,r,mu,1)   # There is only 1 ARG in the replicates iterator
    
    for tree_seq in ts_replicates:
        Nvar = 0
        Nfix_der = 0
        Nfix_anc = 0
        
        x_last = 0
        for variant in tree_seq.variants():
            x = int(variant.site.position)
            
            d = x-x_last
            nanc = np.max([d-1,0])   
            Nfix_anc += nanc
            
            S[x,:] = list(variant.genotypes)
            nd = np.sum(S[x,:])
            if nd == 0:
                Nfix_anc += 1
            else:
                Nvar += 1
                if nd == N:
                    Nfix_der += 1
            x_last = x

    print('Nvar:',Nvar)
    print('Nfix_der',Nfix_der)
    print('Nfix_anc:',Nfix_anc)
    print('Ntot:',Nvar+Nfix_anc)
    
    return S

In [209]:
1.7976931348623157e+300

1.7976931348623156e+300

In [35]:
outofafrica_model_parameters([100,0,0])

920.0


([<msprime.simulations.PopulationConfiguration at 0x7f746606acc0>,
 [[0, 2.5e-05, 7.8e-06], [2.5e-05, 0, 3.11e-05], [7.8e-06, 3.11e-05, 0]],
 [{'type': 'mass_migration', 'time': 920.0, 'source': 2, 'dest': 1, 'proportion': 1.0},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0, 'matrix_index': None},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0.00015, 'matrix_index': (0, 1)},
  {'type': 'migration_rate_change', 'time': 920.0, 'rate': 0.00015, 'matrix_index': (1, 0)},
  {'type': 'population_parameters_change', 'time': 920.0, 'growth_rate': 0, 'initial_size': 1861, 'population': 1},
  {'type': 'mass_migration', 'time': 2040.0, 'source': 1, 'dest': 0, 'proportion': 1.0},
  {'type': 'population_parameters_change', 'time': 5920.0, 'growth_rate': None, 'initial_size': 7300, 'population': 0}])

In [34]:
#This function sets up the out_of_africa model, outlined in the msprime tutorial and expanded by user "slowkoni" on github, from the Gravel 2011 paper
#these values will be used to initialize a simulation so that additional user-definied events can simulated in more recent times
def outofafrica_model_parameters(n_samples):
    generation_time = 25

#     n_samples = [10,10,10]

    m_AF_B = 15e-5
    m_AF_EU = 2.5e-5
    m_AF_AS = 0.78e-5
    m_EU_AS = 3.11e-5

    N_A = 7300 #ancestral_size 
    N_AF = 14474 #africa_size 
    N_EU0 = 1032 #out-to-europe-size
    N_AS0 = 550 #out-to-asia-size
    N_AF_B = 1861 #out-of-africa-size

    T_B = 51000 #merge-to-africa time
    T_EuAs = 23000 #merge-europe-asia-time
    T_AF = 148000 #africa-expansion-time

    asia_growth_rate = 0.38 
    europe_growth_rate = 0.48

     # Calculate the final sizes, or in coalescent terms, the starting sizes, of the
    # Europe and East Asia populations which under the Gravel et al. 2011 out of africa
    # model experience exponential growth after diverging from each other going
    # separate ways in the world. Command line accepts values in percentages as given in
    # the Gravel et al. paper. We just need to know the starting effective size, starting
    # as in the present day effective size, as the simulation goes backward in time.
    europe_final_size = N_EU0 / math.exp(-(europe_growth_rate/100.) * (T_EuAs/generation_time))
    asia_final_size = N_AS0 / math.exp(-(asia_growth_rate/100.) * (T_EuAs/generation_time))

    migration_matrix = [
        [      0, m_AF_EU, m_AF_AS],
        [m_AF_EU,       0, m_EU_AS],
        [m_AF_AS, m_EU_AS,       0],
    ]
    
    europe_asia_merge_time = T_EuAs/generation_time
    print(europe_asia_merge_time)
    population_configurations = [
        ms.PopulationConfiguration(
            sample_size=n_samples[0], initial_size=N_AF),
        ms.PopulationConfiguration(
            sample_size=n_samples[1], initial_size=europe_final_size,
            growth_rate=europe_growth_rate/100.),
        ms.PopulationConfiguration(
            sample_size=n_samples[2], initial_size=asia_final_size,
            growth_rate=asia_growth_rate/100.)
    ]

    demographic_events = [
        # All the next events, until indicated below, are coincident and thus really one
        # event
        
        # The Europe (pop 1) and East Asia (pop 2) populations merge and become population 1
        ms.MassMigration(
            time=europe_asia_merge_time, source=2, destination=1, proportion=1.0),
        # Migration rates must now refelect migration between the original African population
        # and the merged Europe/East Asian population which are still separate at this point
        ms.MigrationRateChange(time=europe_asia_merge_time, rate=0),
        ms.MigrationRateChange(
            time=europe_asia_merge_time, rate=m_AF_B, matrix_index=(0, 1)),
        ms.MigrationRateChange(
            time=europe_asia_merge_time, rate=m_AF_B, matrix_index=(1, 0)),
        # Finally, the effective size of the merged population (pop 1 now) is different
        # otherwise it would be whatever the shrinking Europe population size is/was.
        ms.PopulationParametersChange(
            time=europe_asia_merge_time, initial_size=N_AF_B,
            growth_rate=0, population_id=1),

        # Now we are backward further in time
        # Next, the migrating out of africa population (currently pop 1) joins to the
        # population of origin (pop 0), the african population
        ms.MassMigration(
            time=T_B/generation_time, source=1, destination=0, proportion=1.0),

        # In the final event of the model, the African population (pop 0, the only one left)
        # reduces in size to the ancestral population size until the MRCA (end of the simulation)
        ms.PopulationParametersChange(
            time=T_AF/generation_time,
            initial_size=N_A, population_id=0)
    ]

    return(population_configurations, migration_matrix, demographic_events)


In [54]:
sample_scheme

[4, 4, 4]

In [67]:

def run_msprime_tskit_v1(theta,sample_nums,L,r,mu,R,outname=None,outname_epoch=None):
    ts_replicates = None
    
    Np = len(sample_nums)
#     print('samples to be drawn for populations: {0}'.format(sample_nums))
    # if(args.ooafrica):
    #     init_pop_configs, init_mig, init_demoevents = outofafrica_model_parameters()
    demo_events = []

    if(theta[0][0] == 'ooafrica'):
        init_pop_configs, init_mig, demo_events = outofafrica_model_parameters(theta[0][1])
    else:
        init_pop_configs = [ms.PopulationConfiguration(sample_size=sample_nums[i], initial_size=theta[0][2][0][i]) for i in range(Np)]
        
        if(theta[0][0] == 'mrate'):
            init_mig = theta[0][2][1]
        elif(theta[0][0] == 'mass'):
            init_mig = None
            for m in theta[0][2][1]:
                demo_events.append(ms.MassMigration(time=theta[0][1], source=m[0], destination=m[1], proportion=m[2]))
    # if(args.ooafrica):
    #     ooa_pop_configs, ooa_mig, ooa_demoevents = outofafrica_model_parameters()
    #     for d in ooa_demoevents:
    #         init_demoevents.append(d)
    K = len(theta)  # K = number of epochs
    print(theta)
    if K > 1:
        # There is more than one epoch, so must set the non-initial epochs as demographic events
        # demo_events = []
        for k in range(1,K):
            theta_type,t_k,theta_k = theta[k]
            

            if(theta_type == 'mrate'):
                Ne = theta_k[0]
                mig = theta_k[1]
                for i in range(Np):
                    # Set the Ne
                    demo_events.append(ms.PopulationParametersChange(population=i,time=t_k,initial_size=Ne[i]))
                    print('matrix to add: {0}'.format(mig))

                    # Set the migration rates
                    for j in range(Np):
                        if j!=i:
                            demo_events.append(ms.MigrationRateChange(time=t_k,rate=mig[i,j],matrix_index=tuple([i,j])))
            elif(theta_type == 'mass'):
                Ne = theta_k[0]
                massmig = theta_k[1]
                for m in massmig:
                    demo_events.append(ms.MassMigration(time=t_k, source=m[0], destination=m[1], proportion=m[2]))
            elif(theta_type == 'ooafrica'):
                ooa_pop_configs, ooa_mig, ooa_demoevents = outofafrica_model_parameters(theta[k][1])
                for d in ooa_demoevents:
                    demo_events.append(d)
            else:
                print('Epoch events should only be "mass" or "mrate"!')
                return -1
        # if(args.ooafrica):
        #     ooa_pop_configs, ooa_mig, ooa_demoevents = outofafrica_model_parameters()
        #     for d in ooa_demoevents:
        #         demo_events.append(d)
#         print(init_pop_configs)
        ts_replicates = ms.simulate(
            length=L,
            recombination_rate=r,
            population_configurations=init_pop_configs,
            migration_matrix = init_mig,
            demographic_events=demo_events,
            num_replicates=R,
            mutation_rate = mu
        )
    else:
        print('there is only one epoch, starting now using theta {0}'.format(theta))
        # if(args.ooafrica):
        #     ooa_pop_configs, ooa_mig, ooa_demoevents = outofafrica_model_parameters()
        #     for d in ooa_demoevents:
        #         print(d)
        #         init_demoevents.append(d)
        #     if(init_mig is None): #The only event provided is a mass migration event, and the user specifies the out-of-africa flag, we can use that migration matrix as the initial migration matrix
        #         init_mig = ooa_mig 
        # There is only the initial epoch. 
        if(init_mig is None): #If the first epoch event is a mass migration event, and the out-of-africa is not provided, then there's no migration matrix to provide
            ts_replicates = ms.simulate(
            length=L,
            recombination_rate=r,
            population_configurations=init_pop_configs,
            demographic_events = demo_events,
            num_replicates=R,
            mutation_rate = mu)
        else:
            if(len(demo_events) > 0):
                ts_replicates = ms.simulate(
                length=L,
                recombination_rate=r,
                population_configurations=init_pop_configs,
                demographic_events = demo_events,
                migration_matrix = init_mig,
                num_replicates=R,
                mutation_rate = mu)
            else:
                ts_replicates = ms.simulate(
                    length=L,
                    recombination_rate=r,
                    population_configurations=init_pop_configs,
                    migration_matrix = init_mig,
                    num_replicates=R,
                    mutation_rate = mu
                )
    if(demo_events != None):
        dp = ms.DemographyDebugger(
            population_configurations=init_pop_configs,
            migration_matrix=init_mig,
            demographic_events=demo_events)
        if(outname == None):
            print('Demography History for epoch {0}'.format(outname_epoch))
            dp.print_history(output=sys.stderr)
        else:
            outfile = open('{0}.epoch{1}.demohistory'.format(outname,outname_epoch),'w')
            dp.print_history(output=outfile)
            outfile.close()
    return ts_replicates

In [116]:
new_ids

['ID0', 'ID1', 'ID2', 'ID3', 'ID4', 'ID5', 'ID6', 'ID7', 'ID8', 'ID9']

In [155]:
def out_of_africa():
    # First we set out the maximum likelihood values of the various parameters
    # given in Table 1.
    N_A = 7300
    N_B = 2100
    N_AF = 12300
    N_EU0 = 1000
    N_AS0 = 510
    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_AF = 220e3 / generation_time
    T_B = 140e3 / generation_time
    T_EU_AS = 21.2e3 / generation_time
    # We need to work out the starting (diploid) population sizes based on
    # the growth rates provided for these two populations
    r_EU = 0.004
    r_AS = 0.0055
    N_EU = N_EU0 / math.exp(-r_EU * T_EU_AS)
    N_AS = N_AS0 / math.exp(-r_AS * T_EU_AS)
    # Migration rates during the various epochs.
    m_AF_B = 25e-5
    m_AF_EU = 3e-5
    m_AF_AS = 1.9e-5
    m_EU_AS = 9.6e-5
    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=YRI, 1=CEU and 2=CHB
    # initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=0, initial_size=N_AF),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_EU, growth_rate=r_EU),
        msprime.PopulationConfiguration(
            sample_size=1, initial_size=N_AS, growth_rate=r_AS)
    ]
    migration_matrix = [
        [      0, m_AF_EU, m_AF_AS],
        [m_AF_EU,       0, m_EU_AS],
        [m_AF_AS, m_EU_AS,       0],
    ]
    demographic_events = [
        # CEU and CHB merge into B with rate changes at T_EU_AS
        msprime.MassMigration(
            time=T_EU_AS, source=2, destination=1, proportion=1.0),
        msprime.MigrationRateChange(time=T_EU_AS, rate=0),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(0, 1)),
        msprime.MigrationRateChange(
            time=T_EU_AS, rate=m_AF_B, matrix_index=(1, 0)),
        msprime.PopulationParametersChange(
            time=T_EU_AS, initial_size=N_B, growth_rate=0, population_id=1),
        # Population B merges into YRI at T_B
        msprime.MassMigration(
            time=T_B, source=1, destination=0, proportion=1.0),
        # Size changes to N_A at T_AF
        msprime.PopulationParametersChange(
            time=T_AF, initial_size=N_A, population_id=0)
    ]
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dd = msprime.DemographyDebugger(
        population_configurations=population_configurations,
        migration_matrix=migration_matrix,
        demographic_events=demographic_events)
#     dd.print_history()
#     tree_sequence = msprime.simulate(sample_size=samp_size, Ne=1e5, length=seq_len, recombination_rate=2e-8,mutation_rate=2e-8, 
#                                  random_seed=10)
#     tree_sequence = msprime.simulate(population_configurations=population_configurations,demographic_events=demographic_events,
#                                      Ne=N_AF, length=seq_len, recombination_rate=2e-8,mutation_rate=2e-8,migration_matrix=migration_matrix)
    return dd

In [156]:
test = out_of_africa()

In [158]:
msprime.simulate(test)

TypeError: 'DemographyDebugger' object cannot be interpreted as an integer